In [46]:
import pandas as pd
df = pd.read_csv("Comp1 - Loans.csv")

In [47]:
df.head()

,Currency,Balance Amount,Actual Outstanding in SGD,Maturity Date,Account Number,Interest Recievable,Bucket,IP/IR bucket,Exchange Rate,Base Ccy,Interest flow,IR Amount,Principal + Interest,Bucket (Principal + Interest),Bucket IR
0,USD,"4,83,587.87","15,08,363.34",31-Jan-17,88160000082,25343.9472,8 days to 14 days,Over 3 wks to 1 mth,1.44655,"6,99,534.04","2,681.30","36,661.29","7,02,215.34",Over 3 wks to 1 mth,Over 3 wks to 1 mth
1,USD,"5,59,143.68","8,08,829.30",17-Feb-18,88160000082,25343.9472,Up to 1 Year,Over 3 wks to 1 mth,1.44655,"8,08,829.30","23,396.80",0,"8,32,226.10",Over 3 wks to 1 mth,Over 1 yr to 2 yrs
2,USD,"1,83,000.00","50,47,012.91",05-Jan-17,88160000183,1598.2692,Up to 3 Months,Over 1 mth to 3 mths,1.44655,"2,64,718.65","67,831.71","2,311.98","3,32,550.36",Over 1 mth to 3 mths,Up to 1 week
3,USD,"1,83,000.00","2,64,718.65",16-Aug-17,88160000183,1598.2692,Up to 6 Months,Over 1 mth to 3 mths,1.44655,"2,64,718.65","66,465.05",0,"3,31,183.70",Over 1 mth to 3 mths,Over 6 mths to 1 yr
4,USD,"1,83,000.00","2,64,718.65",27-Jan-18,88160000183,1598.2692,Up to 1 Year,Over 1 mth to 3 mths,1.44655,"2,64,718.65","62,785.95",0,"3,27,504.60",Over 1 mth to 3 mths,Over 1 yr to 2 yrs


In [48]:
# need some data cleaning before identifying datatypes...
# some column names have blank space need to remove it to read easier
df.columns = df.columns.str.strip()

def remove_commas(x): return str(x).replace(',', '')

# Apply the function to all columns of the DataFrame
df = df.applymap(remove_commas)

In [49]:
df.head()

,Currency,Balance Amount,Actual Outstanding in SGD,Maturity Date,Account Number,Interest Recievable,Bucket,IP/IR bucket,Exchange Rate,Base Ccy,Interest flow,IR Amount,Principal + Interest,Bucket (Principal + Interest),Bucket IR
0,USD,483587.87,1508363.34,31-Jan-17,88160000082,25343.9472,8 days to 14 days,Over 3 wks to 1 mth,1.44655,699534.04,2681.30,36661.29,702215.34,Over 3 wks to 1 mth,Over 3 wks to 1 mth
1,USD,559143.68,808829.30,17-Feb-18,88160000082,25343.9472,Up to 1 Year,Over 3 wks to 1 mth,1.44655,808829.30,23396.80,0,832226.10,Over 3 wks to 1 mth,Over 1 yr to 2 yrs
2,USD,183000.00,5047012.91,05-Jan-17,88160000183,1598.2692,Up to 3 Months,Over 1 mth to 3 mths,1.44655,264718.65,67831.71,2311.98,332550.36,Over 1 mth to 3 mths,Up to 1 week
3,USD,183000.00,264718.65,16-Aug-17,88160000183,1598.2692,Up to 6 Months,Over 1 mth to 3 mths,1.44655,264718.65,66465.05,0,331183.70,Over 1 mth to 3 mths,Over 6 mths to 1 yr
4,USD,183000.00,264718.65,27-Jan-18,88160000183,1598.2692,Up to 1 Year,Over 1 mth to 3 mths,1.44655,264718.65,62785.95,0,327504.60,Over 1 mth to 3 mths,Over 1 yr to 2 yrs


In [50]:
from datetime import datetime
from dateutil import parser

max_before_dot = 0
max_after_dot = 0

def get_datatype(element):
    global max_before_dot
    global max_after_dot
    try:
        int_no = int(element)
        return "Integer"
    except ValueError:
        pass

    try:
        float_no = float(element)
        float_string = str(float_no)
        
        before_dot = float_string.split('.')[0]
        after_dot = float_string.split('.')[1]

        if len(before_dot) > max_before_dot:
            max_before_dot = len(before_dot)

        if len(after_dot) > max_after_dot:
            max_after_dot = len(after_dot)

        # print(len(after_dot))
        
        return "Decimal"
    except ValueError:
        pass
        
    try:
        dt_1 = datetime.strptime(element, "%d-%b-%y")
        parsed_date = parser.parse(element)
        # detected_format = parsed_date.strftime('%Y-%m-%d')
        
        for format_string in ["%Y-%m-%d", "%m/%d/%Y", "%d/%m/%Y", "%d-%b-%y", "%B %d, %Y", "%b %dth, %Y"]:
            try:
                datetime.strptime(element, format_string)
                break
            except ValueError:
                pass
  
        return "DateTime :" + format_string
    except ValueError:
        pass
        
        
    try:
        str_1 = str(element)
        return "String"
    except ValueError:
        pass
    
    return "Unknown"

In [51]:
def get_dataframe_datatype(df):
    dd = dict() # creating new dictionary for column and its data type

    for key in df.keys():
        # using set eleminating duplicates in keywords..
        my_series = df[key].apply(lambda x: get_datatype(x))

        # getting more than one datatype like integer and decimals both so fiding counts for each datatype
        occurrence_series = my_series.value_counts()
        
        if 'Decimal' in occurrence_series:
            # print("decimal existing in list....")
            dd[key] = "Decimal(" + str(max_before_dot) + ", " + str(max_after_dot) + ")"
        
        # logic for identifying string-categorical
        elif occurrence_series.index[0] == "String" and len(set(df[key].duplicated())) > 1:
            # set will contain true and false both if duplicated...
            dd[key] = "String-Categorical"
        
        else:
            # generating dict for each column and its datatype
            dd[key] = occurrence_series.index[0]
    return dd

In [52]:
dd = get_dataframe_datatype(df)

In [53]:
dd

{'Currency': 'String-Categorical',
 'Balance Amount': 'Decimal(8, 2)',
 'Actual Outstanding in SGD': 'Decimal(8, 2)',
 'Maturity Date': 'DateTime :%d-%b-%y',
 'Account Number': 'Integer',
 'Interest Recievable': 'Decimal(8, 4)',
 'Bucket': 'String-Categorical',
 'IP/IR bucket': 'String-Categorical',
 'Exchange Rate': 'Decimal(8, 5)',
 'Base Ccy': 'Decimal(8, 5)',
 'Interest flow': 'Decimal(8, 5)',
 'IR Amount': 'Decimal(8, 5)',
 'Principal + Interest': 'Decimal(8, 5)',
 'Bucket (Principal + Interest)': 'String-Categorical',
 'Bucket IR': 'String-Categorical'}

In [54]:
df.head()

,Currency,Balance Amount,Actual Outstanding in SGD,Maturity Date,Account Number,Interest Recievable,Bucket,IP/IR bucket,Exchange Rate,Base Ccy,Interest flow,IR Amount,Principal + Interest,Bucket (Principal + Interest),Bucket IR
0,USD,483587.87,1508363.34,31-Jan-17,88160000082,25343.9472,8 days to 14 days,Over 3 wks to 1 mth,1.44655,699534.04,2681.30,36661.29,702215.34,Over 3 wks to 1 mth,Over 3 wks to 1 mth
1,USD,559143.68,808829.30,17-Feb-18,88160000082,25343.9472,Up to 1 Year,Over 3 wks to 1 mth,1.44655,808829.30,23396.80,0,832226.10,Over 3 wks to 1 mth,Over 1 yr to 2 yrs
2,USD,183000.00,5047012.91,05-Jan-17,88160000183,1598.2692,Up to 3 Months,Over 1 mth to 3 mths,1.44655,264718.65,67831.71,2311.98,332550.36,Over 1 mth to 3 mths,Up to 1 week
3,USD,183000.00,264718.65,16-Aug-17,88160000183,1598.2692,Up to 6 Months,Over 1 mth to 3 mths,1.44655,264718.65,66465.05,0,331183.70,Over 1 mth to 3 mths,Over 6 mths to 1 yr
4,USD,183000.00,264718.65,27-Jan-18,88160000183,1598.2692,Up to 1 Year,Over 1 mth to 3 mths,1.44655,264718.65,62785.95,0,327504.60,Over 1 mth to 3 mths,Over 1 yr to 2 yrs


In [55]:
from dateutil import parser

# Example date strings
date_strings = [
    "2024-03-28",
    "28-03-2024",
    "03/28/2024",
    "28 Mar 2024",
    "March 28, 2024",
    "Mar 28th, 2024",
    "2024/13/28",
    "28-Mer-2024",
    "28th of March, 2024"
]


for date_str in date_strings:
    try:
        parsed_date = parser.parse(date_str)
        print(parsed_date)
        detected_format = parsed_date.strftime('%Y-%m-%d')
        # print(f"Date string: {date_str} => Detected format: {detected_format}")
    except ValueError:
        print(f"Date string: {date_str} => Format detection failed")

2024-03-28 00:00:00
2024-03-28 00:00:00
2024-03-28 00:00:00
2024-03-28 00:00:00
2024-03-28 00:00:00
2024-03-28 00:00:00
Date string: 2024/13/28 => Format detection failed
Date string: 28-Mer-2024 => Format detection failed
2024-03-28 00:00:00
